In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
from prettytable import PrettyTable
from sklearn.decomposition import KernelPCA as kpca
from sklearn.metrics import roc_curve, auc,roc_auc_score,f1_score,confusion_matrix
import numpy as np
print (__doc__)

Automatically created module for IPython interactive environment


### Loading Data

In [2]:
filename="apnea.csv"
f = open(filename)
f.readline()  # skip the header
data =  np.loadtxt(fname = f, delimiter = ',',dtype='double')
Y = data[:,data.shape[1]-1]
X = data[:,1:data.shape[1]-1] 

# Reading the labels now
f= open(filename)
labels_=np.loadtxt(fname=f,delimiter=',',dtype='string')
labels_=labels_[0,1:data.shape[1]-1]

In [3]:
#** Reading File into Numpy Array **
print 'X: ',X
print 'Y: ',Y

X:  [[  1.   1.   3. ...,   0.  31.   1.]
 [  1.   1.   3. ...,   4.  31.   1.]
 [  1.   1.   3. ...,   3.  30.   2.]
 ..., 
 [  1.   3.   4. ...,   1.  29.   1.]
 [  2.   4.   4. ...,   0.  34.   1.]
 [  1.   4.   4. ...,   0.  33.   2.]]
Y:  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  0.  1.  0.  1.  0.  1.  0.  1.  1.  1.  1.  0.  0.  1.
  0.  1.  0.  0.  1.  1.  1.  0.  1.  0.  0.  1.  1.  0.  0.  1.  0.  0.
  0.  1.  1.  1.  0.  0.  0.  0.  1.  0.  1.  0.  1.  0.  1.  0.  0.  1.
  0.  1.  1.  1.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  1.  1.  0.  1.
  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  0.  1.  1.  0.  0.  1.  1.  0.  0.  0.  0.  0.  1.  0.  1.
  1.  0.  1.  0.  0.  1.  0.  0.  1.  1.  1.  1.  0.  1.  1.  1.  0.  0.
  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  

*Normalizing to Zero Mean Unit Variance*

In [4]:
mean = X.mean(axis=0)
std = X.std(axis=0)
X = (X - mean) / std

In [5]:
print labels_.shape[0],' Labels selected are: ',labels_ 

16  Labels selected are:  ['AGASGA' 'STEROIDS' 'DEXABETA' 'DELVMODE' 'APG1MIN' 'APG5MIN' 'RESUSNEED'
 'MODERESUS' 'BWT' 'HC_Birth' 'HRday1' 'HR48HRS' 'HR72HRS'
 'Desaturation1_3DAYS' 'Gestation' 'Sex']


**POST Normalization, we Have:**

In [6]:
# Post Normalization
print 'Normalization gives: ',X

Normalization gives:  [[-0.51483827 -1.99629336  0.15054546 ..., -0.33839133  0.67872844
  -0.96248655]
 [-0.51483827 -1.99629336  0.15054546 ...,  0.7294969   0.67872844
  -0.96248655]
 [-0.51483827 -1.99629336  0.15054546 ...,  0.46252484  0.18766685
   1.03897555]
 ..., 
 [-0.51483827 -0.04353288  1.20101823 ..., -0.07141928 -0.30339474
  -0.96248655]
 [ 1.69967155  0.93284737  1.20101823 ..., -0.33839133  2.1519132
  -0.96248655]
 [-0.51483827  0.93284737  1.20101823 ..., -0.33839133  1.66085161
   1.03897555]]


### Principal Component Analysis of Dataset

In [7]:
A=data
unqA = np.unique(A[:,A.shape[1]-1])
out = {unqA[i]:A[A[:,A.shape[1]-1]==unqA[i],0:A.shape[1]-1] for i in range(len(unqA))}

In [8]:
X_0=out[0][:,1:out[0].shape[1]]
X_1=out[1][:,1:out[1].shape[1]]

In [9]:
print X_0,'\n\n',X_1,'\n\n'

[[  1.   2.   2. ...,   0.  30.   1.]
 [  1.   2.   2. ...,   0.  28.   1.]
 [  1.   4.   3. ...,   0.  32.   1.]
 ..., 
 [  1.   3.   4. ...,   1.  29.   1.]
 [  2.   4.   4. ...,   0.  34.   1.]
 [  1.   4.   4. ...,   0.  33.   2.]] 

[[  1.   1.   3. ...,   0.  31.   1.]
 [  1.   1.   3. ...,   4.  31.   1.]
 [  1.   1.   3. ...,   3.  30.   2.]
 ..., 
 [  1.   4.   4. ...,   0.  32.   2.]
 [  1.   2.   2. ...,   0.  32.   1.]
 [  1.   1.   3. ...,   0.  33.   2.]] 




In [10]:
X_0 = (X_0 - mean) / std
X_1 = (X_1 - mean) / std

print 'X_0: ',X_0,'\nX_1: ',X_1

X_0:  [[-0.51483827 -1.01991312 -0.89992731 ..., -0.33839133  0.18766685
  -0.96248655]
 [-0.51483827 -1.01991312 -0.89992731 ..., -0.33839133 -0.79445632
  -0.96248655]
 [-0.51483827  0.93284737  0.15054546 ..., -0.33839133  1.16979002
  -0.96248655]
 ..., 
 [-0.51483827 -0.04353288  1.20101823 ..., -0.07141928 -0.30339474
  -0.96248655]
 [ 1.69967155  0.93284737  1.20101823 ..., -0.33839133  2.1519132
  -0.96248655]
 [-0.51483827  0.93284737  1.20101823 ..., -0.33839133  1.66085161
   1.03897555]] 
X_1:  [[-0.51483827 -1.99629336  0.15054546 ..., -0.33839133  0.67872844
  -0.96248655]
 [-0.51483827 -1.99629336  0.15054546 ...,  0.7294969   0.67872844
  -0.96248655]
 [-0.51483827 -1.99629336  0.15054546 ...,  0.46252484  0.18766685
   1.03897555]
 ..., 
 [-0.51483827  0.93284737  1.20101823 ..., -0.33839133  1.16979002
   1.03897555]
 [-0.51483827 -1.01991312 -0.89992731 ..., -0.33839133  1.16979002
  -0.96248655]
 [-0.51483827 -1.99629336  0.15054546 ..., -0.33839133  1.66085161
   1

In [15]:
import matplotlib.pylab as pylab

import matplotlib.pyplot as plt
# To getter a better understanding of interaction of the dimensions
# plot the first three PCA dimensions
fig = plt.figure(1, figsize=(18, 10))
ax = Axes3D(fig, elev=-150, azim=110)
# Replace mle with any number of components desired for analysis

PCA_var = PCA(n_components=3)
X_0_red =PCA_var.fit_transform(X_0)
X_1_red =PCA_var.fit_transform(X_1)
print 'The variance contributed by each PC by order is: ',PCA_var.explained_variance_ratio_
print 'MLE Method Parameters Chosen: ',PCA_var.n_components_

ax.scatter(X_0_red[:, 0], X_0_red[:, 1], X_0_red[:, 2], c=np.ones(X_0_red.shape[0]),label='No Apnea',cmap=plt.cm.Greys_r)
ax.scatter(X_1_red[:, 0], X_1_red[:, 1], X_1_red[:, 2], c=np.zeros(X_1_red.shape[0]), marker='x',label='Apnea',cmap=plt.cm.Greys_r)
# for Default colormap: Class 0: Blue Class 1: Red

ax.set_xlabel("1st eigenvector")
ax.w_xaxis.set_ticklabels([])
ax.set_ylabel("2nd eigenvector")
ax.w_yaxis.set_ticklabels([])
ax.set_zlabel("3rd eigenvector")
ax.w_zaxis.set_ticklabels([])
ax.legend(loc="upper right")
plt.show()

The variance contributed by each PC by order is:  [ 0.2113572   0.14540507  0.11550571]
MLE Method Parameters Chosen:  3


### Using varying kernels allows for Non-linear manifold representations

## Kernel PCA

**It's clear from the representation below, that the dataset forms three distinct clusters, that can be segregated using a high-dimensional non-linear classifier**

In [17]:
fig = plt.figure(1, figsize=(18, 10))
ax = Axes3D(fig, elev=-150, azim=110)
PCA_var= kpca(n_components=3,kernel='sigmoid')
PCA_var.fit(X)
# X_reduced=PCA_var.fit_transform(X)
X_0_red =PCA_var.transform(X_0)
X_1_red =PCA_var.transform(X_1)

ax.scatter(X_0_red[:, 0], X_0_red[:, 1], X_0_red[:, 2], c=np.zeros(X_0_red.shape[0]),label='No Apnea',cmap=plt.cm.Greys_r)
ax.scatter(X_1_red[:, 0], X_1_red[:, 1], X_1_red[:, 2], c=np.ones(X_1_red.shape[0]), marker='x',label='Apnea',cmap=plt.cm.Greys_r)
# ax.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=Y,cmap=plt.cm.spectral)
ax.set_xlabel("1st eigenvector")
ax.w_xaxis.set_ticklabels([])
ax.set_ylabel("2nd eigenvector")
ax.w_yaxis.set_ticklabels([])
ax.set_zlabel("3rd eigenvector")
ax.w_zaxis.set_ticklabels([])
ax.legend(loc="upper right")
plt.show()

# Classification

### Defining Confusion Matrix Function

In [ ]:
class_labels_=np.array(['Group 1','Group 2','Group 3'])
def plot_confusion_matrix(cm,name,title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(class_labels_))
    plt.xticks(tick_marks,class_labels_, rotation=45)
    plt.yticks(tick_marks,class_labels_)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

**TRAINING MODELS BELOW: **

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

f = open(filename)
f.readline()  # skip the header
data =  np.loadtxt(fname = f, delimiter = ',',dtype='double')
Y = data[:,0]
X = data[:, 1:data.shape[1]]  # we only take the first two features.
names = ["Nearest Neighbors", "RBF SVM", "Decision Tree",
         "Random Forest", "Naive Bayes"]
classifiers = [
    KNeighborsClassifier(),
    SVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GaussianNB()]


#Normalizing to Zero Mean Unit variance

mean = X.mean(axis=0)
std = X.std(axis=0)
X = (X - mean) / std


# preprocess dataset, split into training and test part
# standardize

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.3)

# iterate over classifiers
for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    y_predic=clf.predict(X_test)
    print 'Classifier: ',name,' Accuracy: ',score
    cm=confusion_matrix(y_test,y_predic)
    np.set_printoptions(precision=2)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure()
    plot_confusion_matrix(cm_normalized,name,title='Normalized confusion matrix for '+name)
    plt.show()
